# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [40]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [41]:
import json
import numpy as np

## Settings / Constants

In [42]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
LLM_PAPER_TITLE = {"Llama70B": "Llama-2-70B", "GPT-3": "GPT-3.5-turbo"}
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment", "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [43]:
FEW_SHOT_STRATEGY = {25: "LRS\\textsubscript{25}",
                     500: "LRS\\textsubscript{500}"}

## Code

In [44]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [47]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n")
    for n_real in [500, 1000, 2000]:
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)
        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"- & - & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")
        else:
            print(
                f"- & - & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

    print("\\hline")
    for llm in LLMS:
        for few_shot_condition in ["fixed", "random"]:
            for freq in SYNTH_COMBINATIONS[few_shot_condition]:
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)
                # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))}"
                else:
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1'], 3))}"
                print(
                    f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

        print("\hline")

aspect_category :



FileNotFoundError: [Errno 2] No such file or directory: '../07 train models/results_json/results_only_real_real500_synth0_aspect_category_random.json'

### Report Metrics Fine-Grained Metrics

#### ACD

In [ ]:
absa_task = "aspect_category"
for n_real in [500, 1000, 2000]:
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)
    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
       for metric in ["f1", "accuracy"]:
           class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"
    print(f"- & - & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for few_shot_condition in SYNTH_COMBINATIONS.keys():
    for llm in LLMS:
        for freq in SYNTH_COMBINATIONS[few_shot_condition]:
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)
            # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
            if absa_task == "TASD":
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])} & {add_thousand_dots(results['eval_f1_macro'])}"
            else:
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])}"

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                   class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"

            print(
                    f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {add_thousand_dots(n_real)} & {add_thousand_dots(n_synth)} & {class_wise_metrics[:-1]} \\\\")
    print("\hline")

- & - & 500 & 0 & 0.769 &0.900 &0.905 &0.920 &0.868 &0.918 &0.588 &0.916 &0.242 &0.937  \\
- & - & 1,000 & 0 & 0.800 &0.910 &0.926 &0.938 &0.902 &0.939 &0.820 &0.951 &0.862 &0.981  \\
- & - & 2,000 & 0 & 0.834 &0.921 &0.934 &0.945 &0.925 &0.952 &0.838 &0.956 &0.900 &0.985  \\
\hline
Llama-2-70B & LRS\textsubscript{500} & 500 & 500 & 0.792 &0.905 &0.904 &0.922 &0.869 &0.921 &0.767 &0.938 &0.866 &0.980  \\
Llama-2-70B & LRS\textsubscript{500} & 500 & 1000 & 0.796 &0.901 &0.884 &0.906 &0.881 &0.928 &0.781 &0.944 &0.855 &0.979  \\
Llama-2-70B & LRS\textsubscript{500} & 500 & 1500 & 0.790 &0.895 &0.850 &0.885 &0.872 &0.922 &0.781 &0.940 &0.795 &0.962  \\
GPT-3.5-turbo & LRS\textsubscript{500} & 500 & 500 & 0.774 &0.896 &0.895 &0.913 &0.880 &0.925 &0.759 &0.936 &0.804 &0.970  \\
GPT-3.5-turbo & LRS\textsubscript{500} & 500 & 1000 & 0.804 &0.902 &0.898 &0.916 &0.891 &0.933 &0.795 &0.945 &0.851 &0.978  \\
GPT-3.5-turbo & LRS\textsubscript{500} & 500 & 1500 & 0.789 &0.890 &0.909 &0.925 &0.891 &

#### ACSA

In [ ]:
absa_task = "aspect_category_sentiment"

idx = 0
for aspect_categories in [["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]:
    print("Table for:", aspect_categories, "\n\n")
    for n_real in [500, 1000, 2000]:
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        condition_string = f"- & - & {add_thousand_dots(n_real)} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for few_shot_condition in SYNTH_COMBINATIONS.keys():
        for llm in LLMS:
            for freq in SYNTH_COMBINATIONS[few_shot_condition]:
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                condition_string = f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {add_thousand_dots(n_real)} & {add_thousand_dots(n_synth)} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


- & - & 500 & 0 & 0.722 & 0.938 & 0.000 & 0.992 & 0.678 & 0.942 & 0.816 & 0.913 & 0.000 & 0.964 & 0.662 & 0.905 \\
- & - & 1000 & 0 & 0.786 & 0.948 & 0.000 & 0.992 & 0.747 & 0.950 & 0.877 & 0.939 & 0.168 & 0.966 & 0.723 & 0.922 \\
- & - & 2000 & 0 & 0.806 & 0.953 & 0.000 & 0.992 & 0.789 & 0.957 & 0.894 & 0.949 & 0.602 & 0.976 & 0.793 & 0.932 \\
\hline
Llama-2-70B & LRS\textsubscript{500} & 500 & 500 & 0.743 & 0.940 & 0.000 & 0.992 & 0.693 & 0.944 & 0.782 & 0.912 & 0.090 & 0.965 & 0.676 & 0.910 \\
Llama-2-70B & LRS\textsubscript{500} & 500 & 1000 & 0.682 & 0.936 & 0.000 & 0.992 & 0.662 & 0.940 & 0.809 & 0.917 & 0.498 & 0.974 & 0.722 & 0.920 \\
Llama-2-70B & LRS\textsubscript{500} & 500 & 1500 & 0.747 & 0.940 & 0.259 & 0.991 & 0.728 & 0.943 & 0.734 & 0.896 & 0.453 & 0.972 & 0.739 & 0.923 \\
GPT-3.5-turbo & LRS\textsubscript{500} & 500 & 500 & 0.766 & 0.942 & 0.250 & 0.992 & 0.744 & 0.950 & 0.825 & 0.920 & 0.567 & 0.977 & 0.714 & 0.918 \\
GPT-3